In [1]:
import pandas as pd
import numpy as np
from surprise import KNNBasic,KNNWithMeans
from surprise import accuracy
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from catboost import CatBoostClassifier
import pandas as pd

In [2]:
visit_data = pd.read_parquet('visit_data.parquet')

In [3]:

visit_data

,user_id,age,gender,name,address,pos1,pos2,user_rating
0,1,18,Мужской,Fox,"Парк Фили, Большая Филёвская улица, 22 ст16а, ...",37.490661,55.750929,5
1,1,18,Мужской,Кремлёвский,"1-я Тверская-Ямская улица, 3 ст1, Москва",37.594379,55.770681,2
2,1,18,Мужской,Soul Burger,"Большой Сухаревский переулок, 25 ст1, Москва",37.632115,55.770884,5
3,2,22,Мужской,МУ-МУ,"Казанский вокзал, Комсомольская площадь, 2, Мо...",37.656479,55.773491,3
4,2,22,Мужской,Москва-Дели,"Ермолаевский переулок, 7, Москва",37.590505,55.764352,5
...,...,...,...,...,...,...,...,...
569467,99999,48,Женский,Центр театра и кино под руководством Никиты Ми...,"Улица Поварская, 33 ст1, Москва",37.586203,55.756832,4
569468,99999,48,Женский,Московский Драматический театр им. М.Н. Ермоловой,"Тверская улица, 5/6, Москва",37.612017,55.757425,5
569469,100000,47,Женский,Кашемир,"ТКЗ Дворец на Яузе, площадь Журавлева, 1, Москва",37.704673,55.784040,5
569470,100000,47,Женский,Мосспектакль,"Улица Покровка, 40Б, Москва",37.652562,55.761082,5


### Добавление искусственной информации о том посетил ли пользователь место или посмотрел подробнее

In [4]:
visit_data['visited'] = np.random.choice([0, 1], size=len(visit_data), p=[0.8, 0.2])
# Просмотр подробной информации (40% вероятность)
visit_data['viewed_detail'] = np.random.choice([0, 1], size=len(visit_data), p=[0.6, 0.4])

In [5]:
visit_data

,user_id,age,gender,name,address,pos1,pos2,user_rating,visited,viewed_detail
0,1,18,Мужской,Fox,"Парк Фили, Большая Филёвская улица, 22 ст16а, ...",37.490661,55.750929,5,0,1
1,1,18,Мужской,Кремлёвский,"1-я Тверская-Ямская улица, 3 ст1, Москва",37.594379,55.770681,2,0,1
2,1,18,Мужской,Soul Burger,"Большой Сухаревский переулок, 25 ст1, Москва",37.632115,55.770884,5,0,1
3,2,22,Мужской,МУ-МУ,"Казанский вокзал, Комсомольская площадь, 2, Мо...",37.656479,55.773491,3,0,1
4,2,22,Мужской,Москва-Дели,"Ермолаевский переулок, 7, Москва",37.590505,55.764352,5,0,0
...,...,...,...,...,...,...,...,...,...,...
569467,99999,48,Женский,Центр театра и кино под руководством Никиты Ми...,"Улица Поварская, 33 ст1, Москва",37.586203,55.756832,4,0,0
569468,99999,48,Женский,Московский Драматический театр им. М.Н. Ермоловой,"Тверская улица, 5/6, Москва",37.612017,55.757425,5,0,0
569469,100000,47,Женский,Кашемир,"ТКЗ Дворец на Яузе, площадь Журавлева, 1, Москва",37.704673,55.784040,5,0,1
569470,100000,47,Женский,Мосспектакль,"Улица Покровка, 40Б, Москва",37.652562,55.761082,5,0,1


### Коллаборативная фильтрация

In [6]:
data = visit_data.copy()

In [7]:
# Подготовка данных для surprise
reader = Reader(rating_scale=(1, 5))  # Рейтинги в вашем датасете от 1 до 5
data = Dataset.load_from_df(data[['user_id', 'name', 'user_rating']], reader)

# Разделение на обучающую и тестовую выборки
trainset, testset = train_test_split(data, test_size=0.25)

### Модель KNN

In [ ]:
# Создание и обучение модели
model = KNNBasic()
model.fit(trainset)

# Предсказания на тестовом наборе
predictions = model.test(testset)

# Расчет RMSE (Root Mean Squared Error)
accuracy.rmse(predictions)

In [ ]:
# Предсказания на тестовом наборе
predictions = model.test(testset)

In [ ]:
from collections import defaultdict
from surprise import accuracy

def precision_recall_at_k(predictions, k=10, threshold=4):
    '''Возвращает словарь с precision и recall по пользователю для заданного k и порога.'''

    # Словарь для хранения результатов
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()

    for uid, user_ratings in user_est_true.items():
        # Сортируем предсказания пользователя по убыванию предсказанного рейтинга
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Число рекомендованных объектов, рейтинг которых выше порога
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings[:k])

        # Число релевантных и рекомендованных объектов
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Число релевантных объектов
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])

        # Precision@K: Процент рекомендованных объектов, которые релевантны
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Процент релевантных объектов, которые были рекомендованы
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

In [ ]:
# Расчет метрик
precisions, recalls = precision_recall_at_k(predictions, k=1, threshold=4)

# Средний precision и recall по всем пользователям
print(f"Precision@5: {sum(prec for prec in precisions.values()) / len(precisions):.4f}")
print(f"Recall@5: {sum(rec for rec in recalls.values()) / len(recalls):.4f}")

In [ ]:
predictions

### Инференс модели

In [ ]:
data = visit_data.copy()
# Подготовка данных для surprise
reader = Reader(rating_scale=(1, 5))  # Рейтинги в вашем датасете от 1 до 5
data = Dataset.load_from_df(data[['user_id', 'name', 'user_rating']], reader)
trainset = data.build_full_trainset()
sim_options = {
    'name': 'pearson_baseline',  # Метрика схожести: косинусное сходство, MSD, pearson, pearson_baseline
    'user_based': True,  # или False, если предпочитаете схожесть на основе объектов
    'min_support': 5,  # Минимальное количество общих объектов/пользователей
    'shrinkage': 0  # Параметр уменьшения для pearson_baseline, 100 по умолчанию
}

# Создание и обучение модели KNNWithMeans
inference_model = KNNWithMeans(k=40, sim_options=sim_options)
cross_validate(inference_model, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
inference_model.fit(trainset)

Estimating biases using als...


In [ ]:
1

In [ ]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
# Допустим, мы хотим сгенерировать рекомендации для пользователя с user_id
# Обучение модели
users = trainset.all_users()
items = trainset.all_items()
item_names = trainset.to_raw_iid  # Получаем исходные идентификаторы объектов

# Предположим, мы хотим сгенерировать рекомендации для пользователя с исходным user_id
user_id = 16  # Пример исходного ID пользователя
inner_user_id = trainset.to_inner_uid(user_id)  # Преобразуем во внутренний ID

# Получаем список объектов, которые пользователь уже оценил
user_rated_items = [item for (item, _) in trainset.ur[inner_user_id]]

# Генерация предсказаний для объектов, которые пользователь не оценил
unrated_items = [item for item in items if item not in user_rated_items]
predictions = [inference_model.predict(inner_user_id, item) for item in unrated_items]

# Сортировка предсказаний по убыванию предсказанного рейтинга
predictions.sort(key=lambda x: x.est, reverse=True)

# Выбор top-N рекомендаций
N = 100
top_n_recommendations = predictions[:N]

# Вывод рекомендаций
print(f"Top-{N} Recommendations for User {user_id}:")
for pred in top_n_recommendations:
    print(f"Item ID: {item_names(pred.iid)}, Predicted Rating: {pred.est}")

In [ ]:
visit_data[visit_data['user_id']==16]

In [ ]:
visit_data[visit_data['name']=='Happy']

In [ ]:
predictions

### Модель SVD

In [ ]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate
data = visit_data.copy()
# Подготовка данных для surprise
reader = Reader(rating_scale=(1, 5))  # Рейтинги в вашем датасете от 1 до 5
data = Dataset.load_from_df(data[['user_id', 'name', 'user_rating']], reader)

# Разделение на обучающую и тестовую выборки
trainset, testset = train_test_split(data, test_size=0.25)

# Использование SVD
algo = SVD()

# Проведение кросс-валидации для оценки модели
cross_validate(algo, trainset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
from surprise.model_selection import GridSearchCV

param_grid = {
    'n_epochs': [5, 10, 20],  # Количество эпох обучения
    'lr_all': [0.002, 0.005],  # Скорость обучения
    'reg_all': [0.4, 0.6]  # Коэффициент регуляризации
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# Лучшие результаты RMSE
print(gs.best_score['rmse'])

# Параметры для лучшего RMSE
print(gs.best_params['rmse'])

In [ ]:
# Обучение модели с лучшими параметрами
algo = SVD(n_epochs=gs.best_params['rmse']['n_epochs'],
           lr_all=gs.best_params['rmse']['lr_all'],
           reg_all=gs.best_params['rmse']['reg_all'])
algo.fit(trainset)

In [ ]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
# Допустим, мы хотим сгенерировать рекомендации для пользователя с user_id = 1
# Обучение модели
users = trainset.all_users()
items = trainset.all_items()
item_names = trainset.to_raw_iid  # Получаем исходные идентификаторы объектов

# Предположим, мы хотим сгенерировать рекомендации для пользователя с исходным user_id
user_id = 16  # Пример исходного ID пользователя
inner_user_id = trainset.to_inner_uid(user_id)  # Преобразуем во внутренний ID

# Получаем список объектов, которые пользователь уже оценил
user_rated_items = [item for (item, _) in trainset.ur[inner_user_id]]

# Генерация предсказаний для объектов, которые пользователь не оценил
unrated_items = [item for item in items if item not in user_rated_items]
predictions = [inference_model.predict(inner_user_id, item) for item in unrated_items]

# Сортировка предсказаний по убыванию предсказанного рейтинга
predictions.sort(key=lambda x: x.est, reverse=True)

# Выбор top-N рекомендаций
N = 100
top_n_recommendations = predictions[:N]

# Вывод рекомендаций
print(f"Top-{N} Recommendations for User {user_id}:")
for pred in top_n_recommendations:
    print(f"Item ID: {item_names(pred.iid)}, Predicted Rating: {pred.est}")

In [ ]:
# Предсказания на тестовом наборе
predictions = algo.test(testset)

In [30]:
# Расчет метрик
precisions, recalls = precision_recall_at_k(predictions, k=1, threshold=4)

# Средний precision и recall по всем пользователям
print(f"Precision@5: {sum(prec for prec in precisions.values()) / len(precisions):.4f}")
print(f"Recall@5: {sum(rec for rec in recalls.values()) / len(recalls):.4f}")

Precision@5: 0.9999
Recall@5: 0.4023


### Гибридная рекомендательная система

In [7]:
from sklearn.model_selection import train_test_split
df = visit_data.copy()
df['name'] = df['name'].astype('category')
df['address'] = df['address'].astype('category')

# Создаем новую колонку 'place_id'
df['place_id'] = df['name'].astype(str) + ' - ' + df['address'].astype(str)
df['place_id'] = df['place_id'].astype('category')

# Преобразование рейтингов и пола пользователя
df['rating_class'] = (df['user_rating'] >= 4).astype(int)
df['gender'] = df['gender'].astype('category')

# Подготовка признаков и целевой переменной
X = df[['user_id', 'age', 'gender', 'place_id']]  # Исключаем ненужные колонки
y = df['rating_class']

# CatBoost умеет работать с категориальными переменными напрямую
cat_features = ['gender', 'place_id']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Инициализация и обучение модели CatBoostClassifier
classifier = CatBoostClassifier(iterations=500,
                                learning_rate=0.05,
                                depth=4,
                                verbose=50,
                                thread_count=-1)
classifier.fit(X_train, y_train, cat_features=cat_features)

0:	learn: 0.6912048	total: 319ms	remaining: 2m 39s
50:	learn: 0.6731845	total: 2.93s	remaining: 25.8s
100:	learn: 0.6730446	total: 4.64s	remaining: 18.4s
150:	learn: 0.6730070	total: 6.48s	remaining: 15s
200:	learn: 0.6729733	total: 8.38s	remaining: 12.5s
250:	learn: 0.6729341	total: 10.2s	remaining: 10.2s
300:	learn: 0.6728915	total: 12.2s	remaining: 8.07s
350:	learn: 0.6728334	total: 14.2s	remaining: 6.03s
400:	learn: 0.6727871	total: 16.2s	remaining: 4.01s
450:	learn: 0.6727398	total: 18.2s	remaining: 1.98s
499:	learn: 0.6726952	total: 20.2s	remaining: 0us


In [8]:
y_test

455577    0
455578    1
455579    0
455580    1
455581    0
         ..
569467    1
569468    1
569469    1
569470    1
569471    1
Name: rating_class, Length: 113895, dtype: int64

In [20]:
from sklearn.metrics import precision_recall_curve, average_precision_score
import numpy as np

# Предсказание вероятностей для тестовой выборки
y_scores = classifier.predict_proba(X_test)[:, 1]

# Расчет precision и recall для всех возможных порогов
precision, recall, thresholds = precision_recall_curve(y_test, y_scores)

# Расчет Average Precision (AP)
ap_score = average_precision_score(y_test, y_scores)


print(f'Average Precision (AP): {ap_score:.4f}')
k = 5
# Опционально: Найти precision и recall при конкретном k, используя пороги
# Например, для порога, соответствующего top-k предсказаний
threshold_k = 0.55
precision_at_k = precision[np.searchsorted(thresholds, threshold_k)]
recall_at_k = recall[np.searchsorted(thresholds, threshold_k)]

print(f'Precision@{k}: {precision_at_k:.4f}, Recall@{k}: {recall_at_k:.4f}')

Average Precision (AP): 0.6007
Precision@5: 0.6000, Recall@5: 0.9333


In [12]:
def prepare_inference_data(user_id, user_features, all_places):
    """
    Создает DataFrame для инференса для одного пользователя с учетом всех неоцененных мест.
    user_id: ID пользователя, для которого генерируются рекомендации
    user_features: Демографические данные пользователя (например, возраст и пол)
    all_places: DataFrame всех возможных мест с их информацией (включая названия и адреса)
    Возвращает DataFrame, подходящий для модели предсказания.
    """
    # Создаем список словарей, где каждый словарь соответствует одной строке в будущем DataFrame
    data_rows = [{
        'user_id': user_id,
        'age': user_features['age'],
        'gender': user_features['gender'],
        'place_id': row['place_id']  # Используем place_id напрямую из all_places
    } for index, row in all_places.iterrows()]

    # Преобразуем список словарей в DataFrame
    data = pd.DataFrame(data_rows)

    return data

# Убедитесь, что all_places содержит уникальные идентификаторы мест (place_id), включая название и адрес
all_places = df[['place_id']].drop_duplicates()

# Подготовка данных для инференса
user_id = 16
user_features = {'age': 25, 'gender': 'Мужской'}
X_infer = prepare_inference_data(user_id, user_features, all_places)

# Теперь X_infer должен содержать корректную структуру для инференса: 
# одна строка для каждой пары "пользователь-место", где место еще не оценено данным пользователем.

In [13]:
X_infer

,user_id,age,gender,place_id
0,16,25,Мужской,"Fox - Парк Фили, Большая Филёвская улица, 22 с..."
1,16,25,Мужской,"Кремлёвский - 1-я Тверская-Ямская улица, 3 ст1..."
2,16,25,Мужской,"Soul Burger - Большой Сухаревский переулок, 25..."
3,16,25,Мужской,"МУ-МУ - Казанский вокзал, Комсомольская площад..."
4,16,25,Мужской,"Москва-Дели - Ермолаевский переулок, 7, Москва"
...,...,...,...,...
2101,16,25,Мужской,"Библиотека №218 - Нежинская улица, 13, Москва"
2102,16,25,Мужской,"библиотека №59 - Ярославское шоссе, 6 к1, Москва"
2103,16,25,Мужской,Российская государственная библиотека - Звёздн...
2104,16,25,Мужской,"Библиотека №27 - Улица Фестивальная, 67, Москва"


In [14]:
# Предполагается, что X_infer - это DataFrame, подготовленный для инференса,
# и predictions - массив вероятностей положительного класса для каждого места
predictions = classifier.predict_proba(X_infer)[:, 1]
# Добавляем вероятности как новый столбец в X_infer
X_infer['probability'] = predictions

# Сортируем места по убыванию вероятности
X_infer_sorted = X_infer.sort_values(by='probability', ascending=False)

# Выбираем топ-N мест
N = 100
top_n_recommendations = X_infer_sorted.head(N)

# Выводим топ-N рекомендаций
print("Топ-{} рекомендаций для пользователя с ID {}:".format(N, user_id))
print(top_n_recommendations[['place_id', 'probability']])

Топ-100 рекомендаций для пользователя с ID 16:
                                              place_id  probability
304               Ra`men - Пятницкая улица, 45, Москва     0.646056
17     Beverly hills diner - Улица Сретенка, 1, Москва     0.644939
281  Petruccio. Pizza&pasta - РЦ Высота, улица Юных...     0.642595
231  Live stars - Красный Октябрь, Берсеневский пер...     0.640918
477         Молодежь - Сущёвская улица, 21 ст8, Москва     0.640697
..                                                 ...          ...
353                Anima - Сущёвская улица, 21, Москва     0.623944
286  Royal estate - ТЦ Estate mall, деревня Обушков...     0.623944
248  Luxor - Посёлок Развилка, ст10, Ленинский горо...     0.623944
185  Вираж - Центральная улица, 98, д. Осеево, Лоси...     0.623944
77   Polyanka bar&kitchen - Улица Большая Полянка, ...     0.623944

[100 rows x 2 columns]


In [15]:
visit_data[visit_data['user_id']==16]

,user_id,age,gender,name,address,pos1,pos2,user_rating,visited,viewed_detail
94,16,19,Мужской,Нах,"Тверская улица, 9, Москва",37.611009,55.759413,5,0,0
95,16,19,Мужской,Петрович,"Улица Мясницкая, 24 ст3, Москва",37.635206,55.762667,5,0,0
96,16,19,Мужской,Весна,"ТЦ Московский, Московский бульвар, 1а, Балаших...",37.960486,55.825886,5,0,0
97,16,19,Мужской,Танzпол,"Шараповская улица, 7, Мытищи, Московская область",37.751242,55.915195,4,0,0
98,16,19,Мужской,Geisha Superman,"Улица Малая Бронная, 2 ст1, Москва",37.598300,55.759026,3,0,0
99,16,19,Мужской,Happy,"Ансамбль Миусского трамвайного депо, улица Лес...",37.593084,55.780218,5,1,1
100,16,19,Мужской,The Бык,"ТДЦ ВЭБ.РФ, Новинский бульвар, 31, Москва",37.582520,55.757830,4,0,0
101,16,19,Мужской,Twenty One Club,"БЦ Обновление Арбата, улица Новый Арбат, 21, М...",37.586733,55.751924,5,0,0


In [16]:
classifier

In [17]:
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(classifier, f)

# Реализация финального подхода- совмещение SVD и CatBoost. Подготовка модели для инференса(обучение модели на всем наборе данных, без деления на test/train)

In [1]:
import pandas as pd
from surprise import SVD, Dataset, Reader
from catboost import CatBoostClassifier, Pool

C:\Users\ASUS\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [47]:
tmp = pd.read_parquet('visit_data.parquet')
tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569472 entries, 0 to 569471
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   user_id      569472 non-null  int64  
 1   age          569472 non-null  int64  
 2   gender       569472 non-null  object 
 3   name         569472 non-null  object 
 4   address      569472 non-null  object 
 5   pos1         569472 non-null  float64
 6   pos2         569472 non-null  float64
 7   user_rating  569472 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 34.8+ MB


In [48]:
# Загрузка данных
data_visit = pd.read_parquet('visits_full.parquet')
data_places = pd.read_parquet('places_full.parquet')
data_users = pd.read_parquet('users_full.parquet')

In [49]:
data_visit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569472 entries, 0 to 569471
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           569472 non-null  int64 
 1   user_id      569472 non-null  object
 2   place_id     569472 non-null  int64 
 3   user_rating  569472 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 17.4+ MB


In [50]:
print(data_places.columns)
print(data_users.columns)
print(data_visit.columns)

Index(['id', 'category', 'subcategory', 'name', 'address', 'rating',
       'reviews_count', 'pos1', 'pos2'],
      dtype='object')
Index(['id', 'age', 'gender'], dtype='object')
Index(['id', 'user_id', 'place_id', 'user_rating'], dtype='object')


In [51]:
data_visit = data_visit.drop('id',axis=1)

In [52]:
data_visit

,user_id,place_id,user_rating
0,1,353,5
1,1,183,2
2,1,233,5
3,2,1,3
4,2,414,5
...,...,...,...
569467,99999,12313,4
569468,99999,2486,5
569469,100000,65770,5
569470,100000,12343,5


In [53]:
data_places.rename(columns={'id':'place_id'},inplace=True)

In [54]:
data = data_visit.merge(data_places,how='left',on='place_id')

In [55]:
data.head()

,user_id,place_id,user_rating,category,subcategory,name,address,rating,reviews_count,pos1,pos2
0,1,353,5,Поесть,Кафе,Fox,"Парк Фили, Большая Филёвская улица, 22 ст16а, ...",4.8,5180 оценок,37.490661,55.750929
1,1,183,2,Поесть,Кафе,Кремлёвский,"1-я Тверская-Ямская улица, 3 ст1, Москва",4.6,267 оценок,37.594379,55.770681
2,1,233,5,Поесть,Кафе,Soul Burger,"Большой Сухаревский переулок, 25 ст1, Москва",2.6,144 оценки,37.632115,55.770884
3,2,1,3,Поесть,Кафе,МУ-МУ,"Охотный ряд, Манежная площадь, 1 ст2, Москва",4.6,894 оценки,37.615036,55.755489
4,2,414,5,Поесть,Кафе,Москва-Дели,"Ермолаевский переулок, 7, Москва",4.7,48 оценок,37.590505,55.764352


In [56]:
data_users = data_users.rename(columns={'id':'user_id'})

In [57]:
data = data.merge(data_users,how='left',on='user_id')

In [59]:
data.head().to_csv('data_visit_20.csv')

In [60]:
# Подготовка данных для обучения SVD
ratings_data = data[['user_id', 'place_id', 'user_rating']]
ratings_data.columns = ['user_id', 'place_id', 'rating']

# Обучение модели SVD
def train_svd(ratings_data):
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(ratings_data[['user_id', 'place_id', 'rating']], reader)

    algo = SVD()
    trainset = data.build_full_trainset()
    algo.fit(trainset)

    user_embeddings = algo.pu
    place_embeddings = algo.qi

    return user_embeddings, place_embeddings

user_embeddings, place_embeddings = train_svd(ratings_data)

# Подготовка данных для обучения CatBoost
train_data = data.copy()

# Фильтруем train_data, оставляя только пользователей и места, присутствующие в ratings_data
train_data = train_data[train_data['user_id'].isin(ratings_data['user_id']) & train_data['place_id'].isin(ratings_data['place_id'])]

# Создаем словарь для сопоставления идентификаторов пользователей с их векторными представлениями
user_embedding_map = {user_id: embedding for user_id, embedding in zip(ratings_data['user_id'], user_embeddings)}

# Получаем векторные представления пользователей из словаря user_embedding_map
train_data['user_embedding'] = train_data['user_id'].map(user_embedding_map)

# Создаем словарь для сопоставления идентификаторов мест с их векторными представлениями
place_embedding_map = {place_id: embedding for place_id, embedding in zip(ratings_data['place_id'], place_embeddings)}

# Получаем векторные представления мест из словаря place_embedding_map
train_data['place_embedding'] = train_data['place_id'].map(place_embedding_map)

# Удаляем строки с отсутствующими значениями user_embedding и place_embedding
train_data.dropna(subset=['user_embedding', 'place_embedding'], inplace=True)

# Преобразование векторных представлений в числовой формат
train_data['user_embedding'] = train_data['user_embedding'].apply(lambda x: list(map(float, x)))
train_data['place_embedding'] = train_data['place_embedding'].apply(lambda x: list(map(float, x)))

# Разделение признаков и целевой переменной
X = train_data[['age', 'gender', 'user_embedding', 'place_embedding', 'category', 'subcategory']]
y = train_data['user_rating']

# Преобразование категориальных признаков
X['gender'] = X['gender'].map({'Мужской': 0, 'Женский': 1})
X['category'] = X['category'].astype('category').cat.codes
X['subcategory'] = X['subcategory'].astype('category').cat.codes

# Преобразование векторных представлений в отдельные признаки
user_embedding_features = pd.DataFrame(X['user_embedding'].tolist(), index=X.index)
user_embedding_features = user_embedding_features.add_prefix('user_embedding_')

place_embedding_features = pd.DataFrame(X['place_embedding'].tolist(), index=X.index)
place_embedding_features = place_embedding_features.add_prefix('place_embedding_')

# Объединение всех признаков
X = pd.concat([X[['age', 'gender', 'category', 'subcategory']], user_embedding_features, place_embedding_features], axis=1)

# Обучение модели CatBoost
model = CatBoostClassifier(iterations=100, depth=4, learning_rate=0.1, loss_function='MultiClass')
model.fit(X, y, cat_features=['gender', 'category', 'subcategory'])

# Сохранение моделей
import pickle

with open('svd_model.pkl', 'wb') as file:
    pickle.dump((user_embeddings, place_embeddings), file)

model.save_model('catboost_model.pkl')

C:\Users\ASUS\AppData\Local\Temp\ipykernel_67064\812377277.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['gender'] = X['gender'].map({'Мужской': 0, 'Женский': 1})
C:\Users\ASUS\AppData\Local\Temp\ipykernel_67064\812377277.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['category'] = X['category'].astype('category').cat.codes
C:\Users\ASUS\AppData\Local\Temp\ipykernel_67064\812377277.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

0:	learn: 1.5807237	total: 753ms	remaining: 1m 14s
1:	learn: 1.5585592	total: 1.08s	remaining: 53.2s
2:	learn: 1.5411722	total: 1.61s	remaining: 52s
3:	learn: 1.5273604	total: 2.04s	remaining: 49s
4:	learn: 1.5162919	total: 2.57s	remaining: 48.8s
5:	learn: 1.5073598	total: 3.21s	remaining: 50.4s
6:	learn: 1.5001349	total: 3.68s	remaining: 48.9s
7:	learn: 1.4942733	total: 4.08s	remaining: 47s
8:	learn: 1.4894701	total: 4.47s	remaining: 45.3s
9:	learn: 1.4855571	total: 4.84s	remaining: 43.6s
10:	learn: 1.4823502	total: 5.32s	remaining: 43s
11:	learn: 1.4797048	total: 5.75s	remaining: 42.2s
12:	learn: 1.4775406	total: 6.12s	remaining: 41s
13:	learn: 1.4757647	total: 6.51s	remaining: 40s
14:	learn: 1.4742952	total: 6.88s	remaining: 39s
15:	learn: 1.4730732	total: 7.33s	remaining: 38.5s
16:	learn: 1.4720672	total: 7.79s	remaining: 38s
17:	learn: 1.4712586	total: 8.14s	remaining: 37.1s
18:	learn: 1.4705318	total: 8.56s	remaining: 36.5s
19:	learn: 1.4699792	total: 8.98s	remaining: 35.9s
20:	l

In [39]:
# Подготовка данных для обучения SVD
ratings_data = data[['user_id', 'name', 'user_rating']]
ratings_data.columns = ['user_id', 'place_id', 'rating']

In [40]:
# Обучение модели SVD
def train_svd(ratings_data):
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(ratings_data[['user_id', 'place_id', 'rating']], reader)

    algo = SVD()
    trainset = data.build_full_trainset()
    algo.fit(trainset)

    user_embeddings = algo.pu
    place_embeddings = algo.qi

    return user_embeddings, place_embeddings

In [41]:
user_embeddings, place_embeddings = train_svd(ratings_data)

In [42]:
user_embeddings

array([[ 0.15206005,  0.14409304,  0.17381017, ..., -0.0613635 ,
        -0.12493406, -0.05060049],
       [-0.08553836,  0.01206799, -0.20408044, ...,  0.03599053,
        -0.15950346, -0.06310973],
       [-0.00772667,  0.02574222, -0.00198568, ..., -0.04719676,
        -0.00570751, -0.10791012],
       ...,
       [ 0.05352427,  0.02377491, -0.21022484, ..., -0.05021269,
        -0.05885577,  0.18780057],
       [-0.15769243,  0.0284799 ,  0.00391285, ...,  0.07500602,
         0.03704788,  0.14761736],
       [ 0.0007596 ,  0.05795188, -0.14249803, ..., -0.01506452,
         0.10180846, -0.05083667]])

In [43]:
data

,user_id,place_id,user_rating,category,subcategory,name,address,rating,reviews_count,pos1,pos2,age,gender
0,1,353,5,Поесть,Кафе,Fox,"Парк Фили, Большая Филёвская улица, 22 ст16а, ...",4.8,5180 оценок,37.490661,55.750929,24,Мужской
1,1,183,2,Поесть,Кафе,Кремлёвский,"1-я Тверская-Ямская улица, 3 ст1, Москва",4.6,267 оценок,37.594379,55.770681,24,Мужской
2,1,233,5,Поесть,Кафе,Soul Burger,"Большой Сухаревский переулок, 25 ст1, Москва",2.6,144 оценки,37.632115,55.770884,24,Мужской
3,2,1,3,Поесть,Кафе,МУ-МУ,"Охотный ряд, Манежная площадь, 1 ст2, Москва",4.6,894 оценки,37.615036,55.755489,20,Мужской
4,2,414,5,Поесть,Кафе,Москва-Дели,"Ермолаевский переулок, 7, Москва",4.7,48 оценок,37.590505,55.764352,20,Мужской
...,...,...,...,...,...,...,...,...,...,...,...,...,...
569467,99999,12313,4,Развлечения,Театры,Центр театра и кино под руководством Никиты Ми...,"Улица Академика Пилюгина, 2, Москва",4.5,3 оценки,37.542338,55.674022,24,Мужской
569468,99999,2486,5,Развлечения,Театры,Московский Драматический театр им. М.Н. Ермоловой,"Тверская улица, 5/6, Москва",4.5,95 оценок,37.612017,55.757425,24,Мужской
569469,100000,65770,5,Развлечения,Театры,Кашемир,"ТКЗ Дворец на Яузе, площадь Журавлева, 1, Москва",NaN,Нет данных,37.704673,55.784040,22,Мужской
569470,100000,12343,5,Развлечения,Театры,Мосспектакль,"Улица Покровка, 40Б, Москва",3.0,3 оценки,37.652562,55.761082,22,Мужской


In [44]:
# Подготовка данных для обучения CatBoost
train_data = data.copy()

# Создаем словарь для сопоставления идентификаторов пользователей с их векторными представлениями
user_embedding_map = {user_id: embedding for user_id, embedding in enumerate(user_embeddings)}

# Получаем векторные представления пользователей из словаря user_embedding_map
train_data['user_embedding'] = train_data['user_id'].map(lambda x: user_embedding_map.get(x))

# Создаем словарь для сопоставления названий мест с их векторными представлениями
place_embedding_map = {place_id: embedding for place_id, embedding in zip(ratings_data['place_id'], place_embeddings)}


In [46]:
train_data

,user_id,place_id,user_rating,category,subcategory,name,address,rating,reviews_count,pos1,pos2,age,gender,user_embedding
0,1,353,5,Поесть,Кафе,Fox,"Парк Фили, Большая Филёвская улица, 22 ст16а, ...",4.8,5180 оценок,37.490661,55.750929,24,Мужской,None
1,1,183,2,Поесть,Кафе,Кремлёвский,"1-я Тверская-Ямская улица, 3 ст1, Москва",4.6,267 оценок,37.594379,55.770681,24,Мужской,None
2,1,233,5,Поесть,Кафе,Soul Burger,"Большой Сухаревский переулок, 25 ст1, Москва",2.6,144 оценки,37.632115,55.770884,24,Мужской,None
3,2,1,3,Поесть,Кафе,МУ-МУ,"Охотный ряд, Манежная площадь, 1 ст2, Москва",4.6,894 оценки,37.615036,55.755489,20,Мужской,None
4,2,414,5,Поесть,Кафе,Москва-Дели,"Ермолаевский переулок, 7, Москва",4.7,48 оценок,37.590505,55.764352,20,Мужской,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569467,99999,12313,4,Развлечения,Театры,Центр театра и кино под руководством Никиты Ми...,"Улица Академика Пилюгина, 2, Москва",4.5,3 оценки,37.542338,55.674022,24,Мужской,None
569468,99999,2486,5,Развлечения,Театры,Московский Драматический театр им. М.Н. Ермоловой,"Тверская улица, 5/6, Москва",4.5,95 оценок,37.612017,55.757425,24,Мужской,None
569469,100000,65770,5,Развлечения,Театры,Кашемир,"ТКЗ Дворец на Яузе, площадь Журавлева, 1, Москва",NaN,Нет данных,37.704673,55.784040,22,Мужской,None
569470,100000,12343,5,Развлечения,Театры,Мосспектакль,"Улица Покровка, 40Б, Москва",3.0,3 оценки,37.652562,55.761082,22,Мужской,None


In [ ]:

# Получаем векторные представления мест из словаря place_embedding_map
train_data['place_embedding'] = train_data['name'].map(lambda x: place_embedding_map.get(x))

# Удаляем строки с отсутствующими значениями user_embedding и place_embedding
train_data.dropna(subset=['user_embedding', 'place_embedding'], inplace=True)

# Преобразование векторных представлений в числовой формат
train_data['user_embedding'] = train_data['user_embedding'].apply(lambda x: list(map(float, x)))
train_data['place_embedding'] = train_data['place_embedding'].apply(lambda x: list(map(float, x)))

# Разделение признаков и целевой переменной
X = train_data[['age', 'gender', 'user_embedding', 'place_embedding']]
y = train_data['user_rating']

# Преобразование категориальных признаков
X['gender'] = X['gender'].map({'Мужской': 0, 'Женский': 1})

# Преобразование векторных представлений в отдельные признаки
user_embedding_features = pd.DataFrame(X['user_embedding'].tolist(), index=X.index)
user_embedding_features = user_embedding_features.add_prefix('user_embedding_')

place_embedding_features = pd.DataFrame(X['place_embedding'].tolist(), index=X.index)
place_embedding_features = place_embedding_features.add_prefix('place_embedding_')

# Объединение всех признаков
X = pd.concat([X[['age', 'gender']], user_embedding_features, place_embedding_features], axis=1)

# Обучение модели CatBoost
model = CatBoostClassifier(iterations=100, depth=4, learning_rate=0.1, loss_function='MultiClass')
model.fit(X, y, cat_features=['gender'])


In [34]:
# Сохранение моделей
import pickle

with open('svd_model.pkl', 'wb') as file:
    pickle.dump((user_embeddings, place_embeddings), file)

model.save_model('catboost_model.pkl')